Load Library

In [1]:
from keras.applications.xception import Xception
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Model
import os
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.callbacks import Callback, ModelCheckpoint,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD,Adam


Load Driver

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load data set

In [3]:
##load data set for subject s
def Load_data(s):
  if s<0 or s>27:
    print('subject not exist')
    return
  else:
    path='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/s'+str(s)
    s_weight='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/s1/s'+str(s)+'.h5'
    train=path+'/'+os.listdir(path)[0]
    test=path+'/'+os.listdir(path)[1]
    X_train=[]
    Y_train=[]
    for x in os.listdir(train):

      f=train+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_train.append(img)
      Y_train.append(label)
    X_test=[]
    Y_test=[]
    for x in os.listdir(test):
      f=test+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_test.append(img)
      Y_test.append(label)
    X_train=np.asarray(X_train,np.float32)
    X_test=np.asarray(X_test,np.float32)
    Y_train=np.asarray(Y_train,np.float32)
    Y_test=np.asarray(Y_test,np.float32)

    num_example=X_train.shape[0]
    arr=np.arange(num_example)
    np.random.shuffle(arr)
    X_train=X_train[arr]
    Y_train=Y_train[arr]
    Y_train = to_categorical(Y_train,num_classes=52)
    Y_test = to_categorical(Y_test,num_classes=52)
    print('successfully load data for subject',str(s))
    print('shape of training data:',X_train.shape)
    print('shape of training label:',Y_train.shape)
    print('shape of test data:',X_test.shape)
    print('shape of test label:',Y_test.shape)
    return X_train,Y_train,X_test,Y_test,s_weight,path


In [4]:
X_train,Y_train,X_test,Y_test,s_weight,path=Load_data(2)

successfully load data for subject 2
shape of training data: (364, 100, 200, 3)
shape of training label: (364, 52)
shape of test data: (156, 100, 200, 3)
shape of test label: (156, 52)


Load model 

In [14]:

base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
x=base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(52, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

overview of performance(skip this part if tuning is required)

In [9]:
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train,
                    Y_train,
                    epochs=64,
                    validation_split=0.1,
                    callbacks=callbacks)


Epoch 1/32
11/11 [==============================] - 19s 847ms/step - loss: 3.0240 - categorical_accuracy: 0.2161 - val_loss: 3.7796 - val_categorical_accuracy: 0.1081


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/32
11/11 [==============================] - 5s 496ms/step - loss: 1.0875 - categorical_accuracy: 0.7299 - val_loss: 5.8359 - val_categorical_accuracy: 0.2973
Epoch 3/32
11/11 [==============================] - 5s 485ms/step - loss: 0.7612 - categorical_accuracy: 0.7914 - val_loss: 7.9703 - val_categorical_accuracy: 0.3514
Epoch 4/32
11/11 [==============================] - 5s 488ms/step - loss: 0.3588 - categorical_accuracy: 0.8937 - val_loss: 9.5006 - val_categorical_accuracy: 0.2703

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/32
11/11 [==============================] - 5s 485ms/step - loss: 0.6165 - categorical_accuracy: 0.8241 - val_loss: 3.8928 - val_categorical_accuracy: 0.4595
Epoch 6/32
11/11 [==============================] - 5s 487ms/step - loss: 0.4222 - categorical_accuracy: 0.9002 - val_loss: 3.2426 - val_categorical_accuracy: 0.5676
Epoch 7/32
11/11 [==============================] - 5s 492ms/step - loss: 0.2294 - catego

In [10]:
model.load_weights(s_weight)
loss, accuracy = model.evaluate(X_test,
                  Y_test)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

5/5 [==============================] - 1s 144ms/step - loss: 0.6228 - categorical_accuracy: 0.8846
loss = 0.6228, accuracy = 0.8846


Fine-tuning

In [ ]:
## if do tuning, plese skip the overview of performance part

In [ ]:
## plot and save model structure 
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/model.png',show_shapes=True)

In [ ]:
model.summary()

In [15]:
## first try tuning last several layer
tuning1_weight='/content/drive/MyDrive/srtp-xsh/RGBimg/raw_image/tuning1_weight.h5'
for layer in base_model.layers:
    layer.trainable = False
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(tuning1_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train,
                    Y_train,
                    epochs=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/32
11/11 [==============================] - 14s 559ms/step - loss: 15.5775 - categorical_accuracy: 0.2274 - val_loss: 12.2149 - val_categorical_accuracy: 0.1892


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/32
11/11 [==============================] - 2s 143ms/step - loss: 6.1493 - categorical_accuracy: 0.3489 - val_loss: 3.8950 - val_categorical_accuracy: 0.3514
Epoch 3/32
11/11 [==============================] - 2s 139ms/step - loss: 1.8884 - categorical_accuracy: 0.5333 - val_loss: 1.7729 - val_categorical_accuracy: 0.5405
Epoch 4/32
11/11 [==============================] - 1s 137ms/step - loss: 1.1560 - categorical_accuracy: 0.6908 - val_loss: 1.7196 - val_categorical_accuracy: 0.5676
Epoch 5/32
11/11 [==============================] - 1s 137ms/step - loss: 0.7922 - categorical_accuracy: 0.7289 - val_loss: 0.9450 - val_categorical_accuracy: 0.7027
Epoch 6/32
11/11 [==============================] - 1s 135ms/step - loss: 0.6522 - categorical_accuracy: 0.7737 - val_loss: 0.6688 - val_categorical_accuracy: 0.7838
Epoch 7/32
11/11 [==============================] - 1s 137ms/step - loss: 0.4319 - categorical_accuracy: 0.8527 - val_loss: 0.6747 - val_categorical_accuracy: 0.7568
Epoc

In [16]:
## then tuning the base model
for layer in model.layers[0:133]:
   layer.trainable = True
for layer in model.layers[133:]:
   layer.trainable = False
model.load_weights(tuning1_weight)
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train,
                    Y_train,
                    epochs=64,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/64
11/11 [==============================] - 20s 833ms/step - loss: 0.2905 - categorical_accuracy: 0.9309 - val_loss: 2.0169 - val_categorical_accuracy: 0.7838


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/64
11/11 [==============================] - 6s 496ms/step - loss: 0.4584 - categorical_accuracy: 0.8831 - val_loss: 1.8565 - val_categorical_accuracy: 0.7568
Epoch 3/64
11/11 [==============================] - 5s 488ms/step - loss: 0.4408 - categorical_accuracy: 0.8844 - val_loss: 1.9734 - val_categorical_accuracy: 0.5676
Epoch 4/64
11/11 [==============================] - 5s 488ms/step - loss: 0.2073 - categorical_accuracy: 0.9298 - val_loss: 1.5919 - val_categorical_accuracy: 0.5946
Epoch 5/64
11/11 [==============================] - 5s 493ms/step - loss: 0.3037 - categorical_accuracy: 0.9080 - val_loss: 3.1862 - val_categorical_accuracy: 0.4324
Epoch 6/64
11/11 [==============================] - 5s 494ms/step - loss: 0.1438 - categorical_accuracy: 0.9635 - val_loss: 2.2043 - val_categorical_accuracy: 0.5676
Epoch 7/64
11/11 [==============================] - 5s 492ms/step - loss: 0.2111 - categorical_accuracy: 0.9329 - val_loss: 1.4851 - val_categorical_accuracy: 0.7297
Epoc

In [17]:
##see performance on test set
model.load_weights(s_weight)
loss, accuracy = model.evaluate(X_test,
                  Y_test)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

5/5 [==============================] - 1s 138ms/step - loss: 0.1789 - categorical_accuracy: 0.9679
loss = 0.1789, accuracy = 0.9679


tuning or fit use generator(use this part, plz skip the above two part)

In [18]:
##generator
def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False, aug=None):
    while 1:  
        assert len(inputs) == len(targets)
        if shuffle:
            indices = np.arange(len(inputs))
            np.random.shuffle(indices)
        for start_idx in range( len(inputs) - batch_size ):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batch_size]
                if aug is not None:
                  (inputs[excerpt], targets[excerpt]) = next(aug.flow(inputs[excerpt],targets[excerpt], batch_size=batch_size))
            else:
                excerpt = slice(start_idx, start_idx + batch_size)
                if aug is not None:
                  (inputs[excerpt], targets[excerpt]) = next(aug.flow(inputs[excerpt],targets[excerpt], batch_size=batch_size))
            yield inputs[excerpt], targets[excerpt]
      

In [19]:
## construct the training image generator for data augmentation to have larger data set
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	horizontal_flip=True, fill_mode="nearest")


In [20]:
## I skip the detailed tuning layer setting here, you can follow the method I use in last part 

In [ ]:
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
his=model.fit_generator(minibatches(X_train,Y_train,batch_size=6,shuffle=False,aug=None),
                            steps_per_epoch=len(X_train)//6,
                            callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True, save_weight_only=True),learning_rate_reduction],
                            validation_data=minibatches(X_test,Y_test,batch_size=6,shuffle=True,aug=None),
                            validation_steps=len(X_1_test)//6,
                            epochs=6)

In [ ]:
##see performance on test set
model.load_weights(s_weight)
loss, accuracy = model.evaluate(X_test,
                  Y_test)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))